In [44]:
import pandas as pd
import geopandas as gpd
import shapely

In [41]:
parks = gpd.read_file('../data/parks/parks-geojson.geojson')
parks

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.96086 1.34618 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.97898 1.38990 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.92313 1.40991 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86697 1.37924 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.82907 1.37276 0.00000)
...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.89036 1.36796 0.00000)
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.98919 1.39017 0.00000)
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88595 1.35520 0.00000)
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.90374 1.39164 0.00000)


In [61]:
parks = parks.to_crs(4326)
parks['geometry'] = parks['geometry'].map(lambda point: shapely.ops.transform(lambda x, y, z=None: (y, x), point))
parks['geometry'] = parks['geometry'].map(lambda point: shapely.ops.transform(lambda x, y: (y, x), point))
parks

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT (103.96086 1.34618)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT (103.97898 1.38990)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT (103.92313 1.40991)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT (103.86697 1.37924)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT (103.82907 1.37276)
...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT (103.89036 1.36796)
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT (103.98919 1.39017)
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT (103.88595 1.35520)
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT (103.90374 1.39164)


In [54]:
# gpd.points_from_xy(y, x)
l = parks['geometry'][0]
[*shapely.ops.transform(lambda x, y, z=None: (y, x), l).coords]

[(103.96085701087, 1.34617500824454)]

In [3]:
from bs4 import BeautifulSoup
import re
from collections import defaultdict

In [15]:
des = parks['Description']

In [16]:
parks_dict = defaultdict(dict)

for i, description in enumerate(des):
    soup = BeautifulSoup(des[i])
    table = soup.find_all('tr')[1:]

    parks_dict[i] = {re.findall('<th>(\w+)<\/th>\s<td>(.+)?<\/td>', str(row))[0][0]:
                           re.findall('<th>(\w+)<\/th>\s<td>(.+)?<\/td>', str(row))[0][1] for row in table}

In [18]:
parks_descriptions = pd.DataFrame(parks_dict).T
parks_descriptions

,ADDRESSBLOCKHOUSENUMBER,ADDRESSBUILDINGNAME,ADDRESSTYPE,HYPERLINK,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,PHOTOURL,ADDRESSPOSTALCODE,DESCRIPTION,ADDRESSSTREETNAME,ADDRESSFLOORNUMBER,INC_CRC,FMEL_UPD_D,ADDRESSUNITNUMBER
0,,,,,42193.6328,36479.08,Jalan Pelatok Open Space,,,Jalan Pelatok,,,BC5EA394BD8CE9F8,20200218182414,
1,,,,,44210.12,41314.54,Turnhouse Park,,,Natheravon Road,,,AB31CCDD36F93C1C,20200218182414,
2,,,,www.nparks.gov.sg/gardens-parks-and-nature/par...,37994.87,43526.7266,Coney Island Park,,,Beside Punggol Promenade Nature Walk,,,7CAC664DFF94240C,20200218182414,
3,,,,,31744.6465,40134.5625,Stratton Walk Playground,,,Located at the end of Stratton Walk,,,8FC92F17CF75A88A,20200218182414,
4,,,,,27527.52,39418.4531,Leban Park,,,Behind Sembawang Hills Food Centre. Upper Tho...,,,EB7459375A3D10F7,20200218182414,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,,,,,34348.5273,38888.0625,Realty Park,,,Along Hougang Ave 4,,,A7BB1BF496807A8B,20200218182414,
346,,,,,45346.4766,41344.2461,Changi Point Promenade,,,Near Lor Bekukong,,,5FEA22BCC48153DA,20200218182414,
347,,,,,33857.832,37476.270000000004,Aroozoo Avenue Playground,,,Bounded by Aroozoo Ave and Jalan Mastuli,,,ACBC9E152D19BD06,20200218182414,
348,,,,,35836.5352,41506.1133,St. Anne's Wood Playground,,,Inside the private housing estate at St Anne's...,,,43A081292A7AE024,20200218182414,


In [33]:
# parks_descriptions.dropna(how='all', axis=1)
import numpy as np
parks_descriptions['ADDRESSUNITNUMBER'].replace('', np.nan, inplace=True)
parks_descriptions

,ADDRESSBLOCKHOUSENUMBER,ADDRESSBUILDINGNAME,ADDRESSTYPE,HYPERLINK,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,PHOTOURL,ADDRESSPOSTALCODE,DESCRIPTION,ADDRESSSTREETNAME,ADDRESSFLOORNUMBER,INC_CRC,FMEL_UPD_D,ADDRESSUNITNUMBER
0,,,,,42193.6328,36479.08,Jalan Pelatok Open Space,,,Jalan Pelatok,,,BC5EA394BD8CE9F8,20200218182414,NaN
1,,,,,44210.12,41314.54,Turnhouse Park,,,Natheravon Road,,,AB31CCDD36F93C1C,20200218182414,NaN
2,,,,www.nparks.gov.sg/gardens-parks-and-nature/par...,37994.87,43526.7266,Coney Island Park,,,Beside Punggol Promenade Nature Walk,,,7CAC664DFF94240C,20200218182414,NaN
3,,,,,31744.6465,40134.5625,Stratton Walk Playground,,,Located at the end of Stratton Walk,,,8FC92F17CF75A88A,20200218182414,NaN
4,,,,,27527.52,39418.4531,Leban Park,,,Behind Sembawang Hills Food Centre. Upper Tho...,,,EB7459375A3D10F7,20200218182414,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,,,,,34348.5273,38888.0625,Realty Park,,,Along Hougang Ave 4,,,A7BB1BF496807A8B,20200218182414,NaN
346,,,,,45346.4766,41344.2461,Changi Point Promenade,,,Near Lor Bekukong,,,5FEA22BCC48153DA,20200218182414,NaN
347,,,,,33857.832,37476.270000000004,Aroozoo Avenue Playground,,,Bounded by Aroozoo Ave and Jalan Mastuli,,,ACBC9E152D19BD06,20200218182414,NaN
348,,,,,35836.5352,41506.1133,St. Anne's Wood Playground,,,Inside the private housing estate at St Anne's...,,,43A081292A7AE024,20200218182414,NaN


In [37]:
parks_descriptions.replace(r'^\s*$', np.nan, regex=True, inplace=True)
parks_descriptions.dropna(axis=1, inplace=True)
parks_descriptions

,LANDXADDRESSPOINT,LANDYADDRESSPOINT,NAME,INC_CRC,FMEL_UPD_D
0,42193.6328,36479.08,Jalan Pelatok Open Space,BC5EA394BD8CE9F8,20200218182414
1,44210.12,41314.54,Turnhouse Park,AB31CCDD36F93C1C,20200218182414
2,37994.87,43526.7266,Coney Island Park,7CAC664DFF94240C,20200218182414
3,31744.6465,40134.5625,Stratton Walk Playground,8FC92F17CF75A88A,20200218182414
4,27527.52,39418.4531,Leban Park,EB7459375A3D10F7,20200218182414
...,...,...,...,...,...
345,34348.5273,38888.0625,Realty Park,A7BB1BF496807A8B,20200218182414
346,45346.4766,41344.2461,Changi Point Promenade,5FEA22BCC48153DA,20200218182414
347,33857.832,37476.270000000004,Aroozoo Avenue Playground,ACBC9E152D19BD06,20200218182414
348,35836.5352,41506.1133,St. Anne's Wood Playground,43A081292A7AE024,20200218182414
